# core

> Main codebase for fetching and saving RSS feeds

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export

In [4]:
#| export
import reader
from typing import *
from bs4 import BeautifulSoup
from rich import print
from rich.panel import Panel
from rich.text import Text
import logging
logger = logging.getLogger(__name__)

def strip_html(s: str):
    s = BeautifulSoup(s)
    return s.text

class PyNewsReader(reader.Reader):
    def __init__(self, dbpath=None, feeds=None):
        if dbpath is None:
            logger.info("Database path not specified, using ./db.sqlite")
            dbpath = "db.sqlite"

        self.reader = reader.make_reader(
            "db.sqlite", 
            plugins=[
                "reader.enclosure_dedupe",
                "reader.entry_dedupe"
            ]
        )
    
        if feeds is None or self.reader.get_feed_counts().total == 0:
            feeds = [
                "https://rss.cbc.ca/lineup/topstories.xml",
                "https://rss.cbc.ca/lineup/world.xml",
                "https://rss.cbc.ca/lineup/canada.xml",
                "https://rss.cbc.ca/lineup/business.xml",
                "https://rss.cbc.ca/lineup/technology.xml",
                "https://www.cbc.ca/cmlink/rss-canada-ottawa",
                "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
                "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
                "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
                "https://www.macleans.ca/feed/",
                "https://thetyee.ca/rss2.xml",
                "https://thenarwhal.ca/feed/",
                "https://ricochet.media/en/feed",
            ]

        for feed in feeds:
            logger.info("Feeds not specified, starting you off with some Canadian news")
            self.reader.add_feed(feed, exist_ok=True)
        
        super().__init__(self.reader._storage, self.reader._search, self.reader._parser, self.reader._reserved_name_scheme)
        self.enable_search()

    def print_entries(self, entries: List[reader.Entry]):
        """
        Pretty print entries - supports reader.Reader.get_entries arguments
        """
        for e in entries:
            if e.published:
                published_date = Text(
                    "Date: " + e.published.isoformat()[:10], justify="center"
                )
            else:
                published_date = Text("Date: Unknown", justify="center")
            print(
                Panel(
                    published_date + "\n" + Text(strip_html(e.summary), justify=None),
                    title=f"[link={e.link}]{e.title}[/link]",
                    subtitle=e.feed.title or e.feed_url,
                )
            )
            print()
    
    def show(self, limit: int = 5, **kwargs) -> type(None):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        self.print_entries(self.get_entries(**kwargs, limit=limit))    
    
    def search(self, query:str):
        """Search entries and pretty print results"""
        entries = []
        for e in r.search_entries(query):
            entries.append(self.search_to_entry(e))
        self.print_entries(entries)
    
    def search_to_entry(self, search_result):
        for i in r.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i
    
    def update(self):
        """Update feeds and search"""
        self.update_feeds()
        self.update_search()


In [5]:
r = PyNewsReader()

/home/mark/mambaforge/lib/python3.10/site-packages/reader/core.py:443: UserWarning: Reader objects should be created using make_reader(); the Reader constructor is not stable yet and may change without any notice.
  warnings.warn(


## List feeds

In [6]:
[i.title or i.url for i in r.get_feeds()]

['https://ricochet.media/en/feed',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans.ca',
 'The Narwhal',
 "The Tyee | BC's home for today's news, analysis and updates",
 'TORONTO STAR',
 'TORONTO STAR',
 'TORONTO STAR']

## Update Feeds

In [7]:
r.update()

## List Entries

In [8]:
r.show(limit=10)

╭──────────────────── ]8;id=918182;https://www.thestar.com/politics/federal/2022/12/07/critics-say-feds-taking-scattergun-approach-to-investment-law-reforms.html\Critics say feds taking ‘scattergun’ approach to investment law reforms]8;;\ ────────────────────╮
│                                                Date: 2022-12-08                                                 │
│             Ottawa unveils heightened scrutiny measures in amendments to the Investment Canada Act              │
╰───────────────────────────────────────────────── TORONTO STAR ──────────────────────────────────────────────────╯

╭─────────────────────── ]8;id=675479;https://www.cbc.ca/news/canada/ottawa/transit-deficit-2022-city-ottawa-safe-restart-1.6678227?cmp=rss\City's transit deficit not fully covered off by provincial grants]8;;\ ───────────────────────╮
│                                                Date: 2022-12-08                                                 │
│   The City of Ottawa wasn't allocated enough funding in a long-awaited provincial announcement to fully cover   │
│                                off its anticipated $85-million transit deficit.                                 │
╰─────────────────────────────────────────────── CBC | Ottawa News ───────────────────────────────────────────────╯

╭────────────────────────────────── ]8;id=851287;https://thetyee.ca/News/2022/12/07/New-Cabinet-Steering-BC/\Meet the New Cabinet Steering BC (in News)]8;;\ ───────────────────────────────────╮
│                                                  Date: Unknown                                                  │
│                         Fresh faces join veterans as Premier David Eby sets priorities.                         │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭──────────────────────────────────── ]8;id=638856;https://thetyee.ca/News/2022/12/07/Mellowing-Laird-Cronk/\The Mellowing of Laird Cronk (in News)]8;;\ ─────────────────────────────────────╮
│                                                  Date: Unknown                                                  │
│           The ex-BC Federation of Labour head on boxing, politics and working with an NDP government.           │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭────────────────────── ]8;id=108898;https://thetyee.ca/Culture/2022/12/07/Visit-Anne-Cameron-In-Tahsis/\At Canada’s End of the Road, a Visit with Anne Cameron (in Culture)]8;;\ ──────────────────────╮
│                                                  Date: Unknown                                                  │
│    The late, noted author (and Tyee commenter) on Indigenous blockades, her most controversial book, life in    │
│                                                Tahsis, and more.                                                │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭───────────────── ]8;id=160614;https://thetyee.ca/Opinion/2022/12/07/Danielle-Smith-As-Queen/\Please Advise! Can Danielle Smith Really Declare Herself Queen? (in Opinion)]8;;\ ──────────────────╮
│                                                  Date: Unknown                                                  │
│           Don’t worry about the premier’s power grab, says Dr. Steve. She’ll change her mind — again.           │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭────────────── ]8;id=420040;https://thetyee.ca/News/2022/12/07/Whistleblower-Protection-For-Crown-Corps/\Whistleblower Protections Expanded to Include BC Housing, Other Agencies (in News)]8;;\ ───────────────╮
│                                                  Date: Unknown                                                  │
│             Some BC Housing employees have alleged the former CEO broke conflict of interest rules.             │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭──────────────────────────── ]8;id=625025;https://thetyee.ca/Analysis/2022/12/06/BC-Hasnt-Learned-Much-Pandemic-Report-Covid-Disappointed/\BC Hasn’t Learned Much from the Pandemic (in Analysis)]8;;\ ─────────────────────────────╮
│                                                  Date: Unknown                                                  │
│        A report on the provincial response to the COVID-19 pandemic shows citizens are disappointed and         │
│                                          disenfranchised. Here’s why.                                           │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭────────────────────── ]8;id=397649;https://thetyee.ca/Culture/2022/12/06/Nan-Goldin-Art-Icon-Big-Pharma-Knees/\Meet the Art Icon Who Brought Big Pharma To Its Knees (in Culture)]8;;\ ───────────────────────╮
│                                                  Date: Unknown                                                  │
│                         Art can change the world, and Nan Goldin’s work does just that.                         │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭─────────────────────── ]8;id=804896;https://thetyee.ca/News/2022/12/06/Calls-For-Flu-Fighting-Vaxapalooza/\Calls for a Flu-Fighting ‘Vaxapalooza.’ But What Else? (in News)]8;;\ ────────────────────────╮
│                                                  Date: Unknown                                                  │
│     Beyond the ‘blitz’ of flu shots aimed for by BC health officials, critics urge more focus on masks and      │
│                                                  ventilation.                                                   │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

## Search Entries

In [9]:
r.search("mars")

╭───────────────────────── ]8;id=51080;https://www.cbc.ca/news/science/mars-moon-occultation-1.6677281?cmp=rss\How you can watch Mars disappear behind the full moon tonight]8;;\ ─────────────────────────╮
│                                                Date: 2022-12-07                                                 │
│  If you happen to have clear skies on Wednesday night, you’ll be able to catch a planet disappearing behind the │
│                                                      moon.                                                      │
╰─────────────────────────────────────────────── CBC | Canada News ───────────────────────────────────────────────╯

╭───────────────────────── ]8;id=923147;https://www.cbc.ca/news/science/mars-moon-occultation-1.6677281?cmp=rss\How you can watch Mars disappear behind the full moon tonight]8;;\ ─────────────────────────╮
│                                                Date: 2022-12-07                                                 │
│  If you happen to have clear skies on Wednesday night, you’ll be able to catch a planet disappearing behind the │
│                                                      moon.                                                      │
╰───────────────────────────────────────────── CBC | Technology News ─────────────────────────────────────────────╯

╭───────────────────────── ]8;id=165417;https://www.cbc.ca/news/science/mars-moon-occultation-1.6677281?cmp=rss\How you can watch Mars disappear behind the full moon tonight]8;;\ ─────────────────────────╮
│                                                Date: 2022-12-07                                                 │
│  If you happen to have clear skies on Wednesday night, you’ll be able to catch a planet disappearing behind the │
│                                                      moon.                                                      │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

╭───────────────────────────────────────── ]8;id=996408;https://www.cbc.ca/news/world/a-second-chance-for-the-bees-1.6676534?cmp=rss\A second chance for the bees]8;;\ ──────────────────────────────────────────╮
│                                                Date: 2022-12-06                                                 │
│     Metalsmith Angel Nieto is an amateur beekeeper in Vina del Mar, Chile, who started receiving calls from     │
│           firefighters last year to take beehives in populated areas and relocate them to safe areas.           │
╰─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╯

In [10]:
## Change title of feed

In [11]:
#| hide
import nbdev

nbdev.nbdev_export()